In [7]:
import pandas as pd 
import openpyxl
import numpy
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [12]:
def transform_df(df, start_year, end_year):

    vars = ['Apr', 'May', 'Jun', 'Jul', 'Aug','Sep','Oct','Nov', 'Dec', 'Jan', "Feb", 'Mar']

    df = pd.melt(df, id_vars=['Region', 'Org Code', 'Modality'], 
               value_vars=vars,var_name='Month', value_name='MedianWaitDays')

    next_year = ['Mar', 'Feb', 'Jan']
    df = df.assign(Year=df['Month'].apply(lambda x: end_year if x in next_year else start_year))

    df['Date'] = pd.to_datetime(df.Month + '-' + df.Year.astype(str), format='%b-%Y')

    df['Date'] = df['Date'].dt.to_period('M')

    return df

# table 2 2020 - 2021

df = pd.read_excel('table2-20-21.xlsx', skiprows=13, sheet_name='Provider')
df = df.iloc[10:1290]
df_20_21 = transform_df(df, 2020, 2021)

# table 2 2021 - 2022

df = pd.read_excel('table2-21-22.xlsx', skiprows=13, sheet_name='Provider')
df = df.iloc[10:1278]
df_21_22 = transform_df(df, 2021, 2022)

# table 2 2022 - 2023
df = pd.read_excel('table2-22-23.xlsx', skiprows=13, sheet_name='Provider')
df = df.iloc[10:1270]
df_22_23 = transform_df(df, 2022, 2023)

# table 2 2023-2024

df = pd.read_excel('table2-23-24.xlsx', skiprows=13, sheet_name='Provider')
df = df.iloc[10:1240]
df_23_24 = transform_df(df, 2023, 2024)


# combine each year into one df 
df = pd.concat([df_20_21, df_21_22, df_22_23, df_23_24])
df = df.dropna()
df = df[df.MedianWaitDays != '.']
df = df[df.MedianWaitDays != '*']

# I think its easier to just focus on 3 modalities, but i'm also okay to do all of them if that's either of your preference. 
# Choosing PET, MRI & CT because those were mentioned in proposal

modalities = ['Magnetic Resonance Imaging', 'Positron Emission Tomography', 'Computerized Axial Tomography']
years = [2019, 2020, 2021]
df = df[df['Modality'].apply(lambda x: any(m in x for m in modalities))]
df = df[df['Year'].apply(lambda x: x>=2020)]
df = df[df['Year'].apply(lambda x: x<2023)]
print(df)

df.to_csv("2020_to_2024_Modalities_Filtered.csv", index=True)

      Region Org Code                       Modality Month MedianWaitDays  \
0        Y56      NT9  Computerized Axial Tomography   Apr              1   
2        Y56      NT9     Magnetic Resonance Imaging   Apr              8   
4        Y56      NT9   Positron Emission Tomography   Apr              7   
5        Y56      RF4  Computerized Axial Tomography   Apr              0   
8        Y56      RF4     Magnetic Resonance Imaging   Apr             17   
...      ...      ...                            ...   ...            ...   
11322    Y63      RFR     Magnetic Resonance Imaging   Dec              6   
11326    Y63      NN4  Computerized Axial Tomography   Dec            5.5   
11329    Y63      NN4     Magnetic Resonance Imaging   Dec             13   
11332    Y63      RCB  Computerized Axial Tomography   Dec              0   
11335    Y63      RCB     Magnetic Resonance Imaging   Dec             20   

       Year     Date  
0      2020  2020-04  
2      2020  2020-04  
4     